In [1]:
# Importing libraries
import cv2 
import numpy as np

In [6]:
class Board:
    
    def __init__(self):
        self.mouse_pressed = False
        self.board = np.zeros(shape=(1024, 1024, 3), dtype=np.uint8)
        self.prev= np.zeros(shape=(1024, 1024, 3), dtype=np.uint8)
        self.char_color = (255, 255, 255)
        self.count= -1
        
    def reset(self):
        # reset image
        self.board = np.zeros((1024, 1024, 3), np.uint8)
        
    def draw(self):
        """
        Method to draw multiple Charactes on a single Board
        """

        self.reset()

        window_name = 'Draw'

        cv2.namedWindow(winname=window_name)
        cv2.setMouseCallback(window_name=window_name, on_mouse=self.mouse_callback)

        # Until 'ESC' key is pressed
        while True:
            cv2.imshow(winname=window_name, mat=self.img)

            # When ESC key pressed
            k = cv2.waitKey(delay=1) & 0xFF
            if k == 27:
                break

        cv2.destroyAllWindows()
        
    

In [7]:
if __name__ == '__main__':
    Board().draw()

AttributeError: 'Board' object has no attribute 'mouse_callback'